In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_24weeks_control_female = sc.read_h5ad('../../exported_data/pvat_24weeks_control_female.h5ad')
pvat_24weeks_control_female

AnnData object with n_obs × n_vars = 7534 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered', 

In [3]:
pvat_24weeks_control_female.obs['celltype_broad'].unique().tolist()

['Adipocytes',
 'Endothelial_Cells',
 'Immune_Cells',
 'Pericytes',
 'Fibroblasts',
 'Mesothelial_Cells',
 'Neuronal_Cells',
 'SMCs']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_24weeks_control_female.raw = pvat_24weeks_control_female
pvat_24weeks_control_female.raw.X

<7534x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 11084559 stored elements in Compressed Sparse Row format>

In [6]:
# Run cellchat analysis on PVAT data
cellchat(pvat_24weeks_control_female, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellchat_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
1774 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.21 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 7534 samples and 18969 features


100%|████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:55<00:00,  8.69it/s]


In [7]:
ligand_receptor_pvat = pvat_24weeks_control_female.uns['cellchat_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
1267,Vegfa,Vegfa,0.954867,0.022260,292.0,Nrp1,Nrp1,0.769832,0.011130,Adipocytes,Endothelial_Cells,0.000495,0.0
83,Vegfa,Vegfa,0.954867,0.022260,292.0,Nrp1,Nrp1,0.806702,0.011130,Adipocytes,Adipocytes,0.000495,0.0
9208,Ntf3,Ntf3,0.813953,0.008990,292.0,Ntrk3,Ntrk3,0.906977,0.025685,SMCs,SMCs,0.000462,0.0
8663,Ncam1,Ncam1,0.636364,0.002568,292.0,Cacna1c,Cacna1c,0.976744,0.051798,Neuronal_Cells,SMCs,0.000266,0.0
665,C4a,C4a,0.910714,0.011344,292.0,Nrp1,Nrp1,0.806702,0.011130,Mesothelial_Cells,Adipocytes,0.000252,0.0


In [8]:
ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellchat_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals]

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
279,Col4a1,Col4a1,0.687075,0.008562,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000015,0.0
304,Col5a3,Col5a3,0.759184,0.008562,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000015,0.0
310,Col3a1,Col3a1,0.744218,0.006849,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000012,0.0
285,Col4a2,Col4a2,0.650340,0.005137,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000009,0.0
273,Col6a6,Col6a6,0.617687,0.005137,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000009,0.0
289,Col6a2,Col6a2,0.738776,0.004281,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000007,0.0
291,Col6a3,Col6a3,0.662585,0.004281,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000007,0.0
287,Col6a1,Col6a1,0.617687,0.003425,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000006,0.0
306,Col5a2,Col5a2,0.613605,0.003425,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000006,0.0
308,Col5a1,Col5a1,0.636735,0.003425,292.0,Itgb1,Itga1_Itgb1,0.362662,0.000856,Fibroblasts,Adipocytes,0.000006,0.0


In [9]:
ligand_receptor_pvat.to_csv("pvat_24weeks_control_female_cellchat.csv")
pvat_24weeks_control_female.write("../../exported_data/pvat_24weeks_control_female.h5ad")